In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format

In [2]:
dados = pd.read_csv(r"C:\Users\marci\OneDrive\CEASA\Fechamento 2023\dados2023 sem matricula.csv", sep=";",decimal=",",
                    thousands=".",
                    parse_dates=['data'])
# dados['data'] = dados['data'].astype('datetime64')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\marci\\OneDrive\\CEASA\\Fechamento 2023\\dados2023 sem matricula.csv'

In [64]:
#Filtrando o Estado para apenas RS
df = dados.loc[dados['estado'] == "RIO GRANDE DO SUL"].copy(deep=True)

#Deletando as colunas desnecessárias
df.drop(columns=['pais','estado','destino','nota_fiscal','unidade','quantidade','peso_kg','cotação','produto','volume'], inplace=True)

# Filtrando os dados para apenas Produtores do RS
df = df.loc[df['atac_gnp'] == 'GNP']

df['data'] = pd.to_datetime(df['data'])
df['mes'] = df['data'].dt.month
meses_do_ano_nomes = {1:"Jan", 2:"Fev",3:"Mar",4:"Abr",5:"Mai",6:"Jun",7:"Jul",8:"Ago",9:"Set",10:"Out",11:"Nov",12:"Dez"}
df["mes"] = df["mes"].map(meses_do_ano_nomes)

# Selecionando as colunas necessárias
df = df[['mes','procedencia','valor comercializado']]

# Agrupando os dados pelo mês e procedência
df = df.groupby(['procedencia','mes'])['valor comercializado'].sum().reset_index()

In [65]:
# Pivotando a tabela
df = df.copy(deep=True)
df = df.pivot(index='procedencia', columns='mes', values='valor comercializado')

# Ajustando a ordem das colunas
df = df[['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']]

# Criando uma Coluna de Totais
df['Total'] = df[['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']].sum(axis=1)

# Melhorando a estilização da tabela
df = df.fillna(0)
df = df.style.format(precision=2, thousands=".", decimal=",").background_gradient(cmap='Greens', axis=1)
df.to_excel(r"C:\Users\marci\OneDrive\CEASA\Fechamento 2023\2023 ICMS SEFAZ RS.xlsx")